In [ ]:
import os
import glob
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
paths = sorted(Path("data/state").glob("*.csv"))
N = len(paths)
T = len(pd.read_csv(paths[0], header=None)) - 1
n = 4
m = 2
p = 1  # lat accel

In [ ]:
targets = np.empty((N, n))
states = np.empty((N, T+1, n))
actions = np.empty((N, T, m))
auxs = np.empty((N, T, p))
for i, path in enumerate(paths):
    target_state_str = path.stem
    target = ([float(v) for v in target_state_str.split("_")])
    target[2] *= (np.pi / 180)
    targets[i] = target
    state = pd.read_csv(Path("data/state") / f"{target_state_str}.csv", header=None).to_numpy()
    action = pd.read_csv(Path("data/action") / f"{target_state_str}.csv", header=None).to_numpy()
    states[i] = state
    actions[i] = action
    speed = state[0:T, 3]
    curvature = action[0:T, 1]
    lat_accel = curvature * (speed ** 2)
    aux = lat_accel[:, None]
    auxs[i] = aux

In [ ]:
plt.figure(figsize=(12, 4))
cmap = matplotlib.colormaps.get("turbo")

for target, state_arr, action_arr, aux_arr in zip(targets, states, actions, auxs):
    # # terminal speed near 10.0
    # if not abs(target[3] - 10.0) < 0.1:
    #     continue
    # # distance x near 10.0
    # if not abs(target[0] - 10.0) < 0.1:
    #     continue
    # if not abs(target[2] - 0.0) < 0.1:
    #     continue

    # x = target[0]
    # y = target[1]
    # v = target[3]
    # color = cmap(0.5 + 0.5 * ((v - 10.0) / 3.0))
    # color = cmap(y / 3.0)


    z_lat = np.percentile(np.abs(aux_arr[:, 0]), q=75) / 6.0
    z_lon = np.percentile(np.abs(action_arr[:, 0]), q=75) / 3.0

    # # lat accel
    # z = z_lat
    
    # # lon accel
    # z = z_lon

    # combined accel, mellowmax
    z = np.log(np.mean(np.exp([z_lat, z_lon])))

    color = cmap(np.clip(z, 0, 1))
    # color = np.clip([z_lat, 0, z_lon], 0, 1)

    plt.plot(state_arr[:, 0], state_arr[:, 1], c=color, lw=1, zorder=1, markersize=4)

plt.scatter(targets[:, 0], targets[:, 1], c='k', s=10, alpha=0.7, zorder=2)
plt.xticks(np.arange(12+1, step=0.4), rotation=45)
# plt.axis("equal")
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
cmap = matplotlib.colormaps.get("turbo")

plt.axhline(-3.0, c="r")
plt.axhline(3.0, c="r")

for target, state_arr, action_arr in zip(targets, states, actions):
    a = action_arr[:, 0]
    
    # # DEBUG filter for high acceleration reversals
    # if np.sum(np.abs(np.diff(np.diff(a))) > 0.8) < 3:
    #     continue
    # print(target)

    plt.plot(a, c="k", lw=1, zorder=1)

# plt.ylim((2.9, 3.1))
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
cmap = matplotlib.colormaps.get("turbo")

plt.axhline(-6.0, c="r")
plt.axhline(6.0, c="r")

for target, aux_arr in zip(targets, auxs):
    plt.plot(aux_arr[:, 0], c="k", lw=1, zorder=1)

# plt.ylim((5.9, 6.1))
plt.grid()
plt.show()

In [ ]:
# for action_arr in actions:
#     plt.plot(action_arr[:, 1], c="k", lw=0.5)
# plt.show()

In [ ]:
# states = pd.read_csv("state.csv", header=None)
# states.columns = ["x", "y", "yaw"]
# actions = pd.read_csv("action.csv", header=None)
# actions.columns = ["speed", "steering"]

In [ ]:
# plt.figure(figsize=(10, 2))
# for state in states.to_numpy():
#     x = state[0]
#     y = state[1]
#     dx = 0.5 * np.cos(state[2])
#     dy = 0.5 * np.sin(state[2])
#     plt.arrow(x, y, dx, dy, width=0.05, color="k", edgecolor=None)
# plt.axis("equal")
# plt.grid()
# plt.show()

In [ ]:
# fig, axs = plt.subplots(nrows=2, sharex=True)
# axs[0].plot(actions.index, actions["speed"])
# axs[1].plot(actions.index, actions["steering"])
# plt.show()